![learn-data-engineering](https://static-media.hotmart.com/NlvE9cql5MgqX2jBxM7BjiMUlL8=/0x60/https://uploads.teachablecdn.com/attachments/7hdFyMPwSMuYp7QIQg6u_lde_logo.png)

In [1]:
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [3]:
spark.sparkContext.setLogLevel("ERROR")  # or "WARN"
spark

In [4]:
# List all Spark configs related to Iceberg catalogs
for k, v in spark.sparkContext.getConf().getAll():
    if k.startswith("spark.sql.catalog."):
        print(f"{k} = {v}")

spark.sql.catalog.demo.s3.endpoint = http://minio:9000
spark.sql.catalog.demo.warehouse = s3://warehouse/wh/
spark.sql.catalog.demo.io-impl = org.apache.iceberg.aws.s3.S3FileIO
spark.sql.catalog.demo.uri = http://rest:8181
spark.sql.catalog.demo.type = rest
spark.sql.catalog.demo = org.apache.iceberg.spark.SparkCatalog


In [5]:
%%sql
SHOW CATALOGS

catalog
demo
spark_catalog


In [6]:
!ls /home/iceberg/data/

customer.parquet  orders.parquet    pokemon.csv
lineitem.parquet  part.parquet	    region.parquet
nation.parquet	  partsupp.parquet  supplier.parquet


In [7]:
%%sql
CREATE DATABASE IF NOT EXISTS academy

++
||
++
++

In [8]:
%%sql
SHOW DATABASES

namespace
academy


In [9]:
schema = StructType([
    StructField("Number", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Type_1", StringType(), True),
    StructField("Type_2", StringType(), True),
    StructField("Abilities", StringType(), True),
    StructField("HP", IntegerType(), True),
    StructField("Att", IntegerType(), True),
    StructField("Def", IntegerType(), True),
    StructField("Spa", IntegerType(), True),
    StructField("Spd", IntegerType(), True),
    StructField("Spe", IntegerType(), True),
    StructField("BST", IntegerType(), True),
    StructField("Mean", DoubleType(), True),
    StructField("Standard_Deviation", DoubleType(), True),
    StructField("Generation", IntegerType(), True),
    StructField("Experience_type", StringType(), True),
    StructField("Experience_to_level_100", IntegerType(), True),
    StructField("Final_Evolution", DoubleType(), True),
    StructField("Catch_Rate", IntegerType(), True),
    StructField("Legendary", DoubleType(), True),
    StructField("Mega_Evolution", DoubleType(), True),
    StructField("Alolan_Form", DoubleType(), True),
    StructField("Galarian_Form", IntegerType(), True),
    StructField("Against_Normal", DoubleType(), True),
    StructField("Against_Fire", DoubleType(), True),
    StructField("Against_Water", DoubleType(), True),
    StructField("Against_Electric", DoubleType(), True),
    StructField("Against_Grass", DoubleType(), True),
    StructField("Against_Ice", DoubleType(), True),
    StructField("Against_Fighting", DoubleType(), True),
    StructField("Against_Poison", DoubleType(), True),
    StructField("Against_Ground", DoubleType(), True),
    StructField("Against_Flying", DoubleType(), True),
    StructField("Against_Psychic", DoubleType(), True),
    StructField("Against_Bug", DoubleType(), True),
    StructField("Against_Rock", DoubleType(), True),
    StructField("Against_Ghost", DoubleType(), True),
    StructField("Against_Dragon", DoubleType(), True),
    StructField("Against_Dark", DoubleType(), True),
    StructField("Against_Steel", DoubleType(), True),
    StructField("Against_Fairy", DoubleType(), True),
    StructField("Height", DoubleType(), True),
    StructField("Weight", DoubleType(), True),
    StructField("BMI", DoubleType(), True)
])

df = spark.read.schema(schema).csv("/home/iceberg/data/pokemon.csv", header=True)

df.writeTo("academy.pokemon") \
  .using("iceberg") \
  .partitionedBy(f.col("Type_1")) \
  .createOrReplace()

In [10]:
%%sql
select max(BST) from academy.pokemon group by Type_1

max(BST)
700
580
610
680
770
780
680
770
690
720


In [11]:
%%sql
SELECT * FROM academy.pokemon.files

+---------+------------------------------------------------------------------------------------------------------------------+-------------+---------+------------------------+--------------+--------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
%%sql
SELECT COUNT(*) FROM academy.pokemon.partitions;

count(1)
18


In [13]:
%%sql
SELECT * from academy.pokemon where HP = 45

Number,Name,Type_1,Type_2,Abilities,HP,Att,Def,Spa,Spd,Spe,BST,Mean,Standard_Deviation,Generation,Experience_type,Experience_to_level_100,Final_Evolution,Catch_Rate,Legendary,Mega_Evolution,Alolan_Form,Galarian_Form,Against_Normal,Against_Fire,Against_Water,Against_Electric,Against_Grass,Against_Ice,Against_Fighting,Against_Poison,Against_Ground,Against_Flying,Against_Psychic,Against_Bug,Against_Rock,Against_Ghost,Against_Dragon,Against_Dark,Against_Steel,Against_Fairy,Height,Weight,BMI
679,Honedge,Steel,Ghost,['No Guard'],45,80,100,35,37,28,325,54.16666666666666,26.453838200827413,None,Medium Fast,1000000,0.0,180,0.0,0.0,0.0,None,0.0,2.0,1.0,1.0,0.5,0.5,0.0,0.0,2.0,0.5,0.5,0.25,0.5,2.0,0.5,2.0,0.5,0.5,0.8,2.0,3.1
225,Delibird,Ice,Flying,"['Hustle', 'Insomnia', 'Vital Spirit']",45,55,45,65,45,75,330,55.0,11.547005383792516,None,Fast,800000,1.0,45,0.0,0.0,0.0,None,1.0,2.0,1.0,2.0,0.5,1.0,1.0,1.0,0.0,1.0,1.0,0.5,4.0,1.0,1.0,1.0,2.0,1.0,0.9,16.0,19.8
238,Smoochum,Ice,Psychic,"['Forewarn', 'Hydration', 'Oblivious']",45,30,15,85,65,65,305,50.833333333333336,23.52599035582184,None,Medium Fast,1000000,0.0,45,0.0,0.0,0.0,None,1.0,2.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,2.0,2.0,2.0,1.0,2.0,2.0,1.0,0.4,6.0,37.5
239,Elekid,Electric,None,"['Static', 'Vital Spirit']",45,63,37,65,55,95,360,60.0,18.43004792903878,None,Medium Fast,1000000,0.0,45,0.0,0.0,0.0,None,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,0.6,23.5,65.3
403,Shinx,Electric,None,"['Guts', 'Intimidate', 'Rivalry']",45,65,34,40,34,45,263,43.833333333333336,10.4788146064121,None,Medium Slow,1059860,0.0,235,0.0,0.0,0.0,None,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,0.5,9.5,38.0
522,Blitzle,Electric,None,"['Lightning Rod', 'Motor Drive', 'Sap Sipper']",45,60,32,50,32,76,295,49.16666666666666,15.517910798672467,None,Medium Fast,1000000,0.0,190,0.0,0.0,0.0,None,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,0.8,29.8,46.6
228,Houndour,Dark,Fire,"['Early Bird', 'Flash Fire', 'Unnerve']",45,60,30,80,50,65,330,55.0,15.811388300841896,None,Slow,1250000,0.0,120,0.0,0.0,0.0,None,1.0,0.5,2.0,1.0,0.5,0.5,2.0,1.0,2.0,1.0,0.0,1.0,2.0,0.5,1.0,0.5,0.5,1.0,0.6,10.8,30.0
624,Pawniard,Dark,Steel,"['Defiant', 'Inner Focus', 'Pressure']",45,85,70,40,40,60,340,56.66666666666666,16.749792701868152,None,Medium Fast,1000000,0.0,120,0.0,0.0,0.0,None,0.5,2.0,1.0,1.0,0.5,0.5,4.0,0.0,2.0,0.5,0.0,1.0,0.5,0.5,0.5,0.5,0.5,1.0,0.5,10.2,40.8
859,Impidimp,Dark,Fairy,"['Frisk', 'Pickpocket', 'Prankster']",45,45,30,55,40,50,265,44.16666666666666,7.861650943380503,None,Medium Fast,1000000,0.0,255,0.0,0.0,0.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.5,0.0,0.25,2.0,2.0,0.4,5.5,34.4
240,Magby,Fire,None,"['Flame Body', 'Vital Spirit']",45,75,37,70,55,83,365,60.833333333333336,16.47641411904369,None,Medium Fast,1000000,0.0,45,0.0,0.0,0.0,None,1.0,0.5,2.0,1.0,0.5,0.5,1.0,1.0,2.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5,0.7,21.4,43.7
